In [3]:
from template import *

In [4]:
def softmax(x):
    # Berechnung des Softmax für jede Zeile der Eingabematrix
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    softmax_x = exp_x / exp_x.sum(axis=1, keepdims=True)
    return softmax_x

softmax(np.array([[-1., 0.], [0.2, 1.]])) 

array([[0.26894142, 0.73105858],
       [0.31002552, 0.68997448]])

In [5]:
np.random.seed(4321)
q = np.random.rand(3,2)
k = np.random.rand(3,2)
v = np.random.rand(3,2)
x = attention(q, k, v)
print(x)

#      [[0.37285946 0.73278279]
#      [0.36712163 0.72522747]
#      [0.36637032 0.72842298]]

[[0.37285946 0.73278279]
 [0.36712163 0.72522747]
 [0.36637032 0.72842298]]


In [6]:

def masked_attention(Q, K, V, mask):
    M = ((Q@np.transpose(K)) / np.sqrt(K.shape[1]) + mask) ## d_k = dim von K
    a = softmax(M) @ V
    return a

In [7]:
np.random.seed(4321)
nf = 10
q = np.random.rand(nf,2)
k = np.random.rand(nf,2)
v = np.random.rand(nf,2)
mask = (1 - np.tri(nf)) * -1e10
x = masked_attention(q, k, v, mask)
print(x)

# [[0.37646796 0.24378126]
#  [0.48299578 0.28439644]
#  [0.46590072 0.41837738]
#  [0.52991302 0.51314059]
#  [0.49214214 0.55574465]
#  [0.39568092 0.59955323]
#  [0.38462954 0.61108759]
#  [0.37248739 0.5645996 ]
#  [0.35915127 0.57331419]
#  [0.41913397 0.51187079]]

[[0.37646796 0.24378126]
 [0.48299578 0.28439644]
 [0.46590072 0.41837738]
 [0.52991302 0.51314059]
 [0.49214214 0.55574465]
 [0.39568092 0.59955323]
 [0.38462954 0.61108759]
 [0.37248739 0.5645996 ]
 [0.35915127 0.57331419]
 [0.41913397 0.51187079]]


In [8]:
def linear_projection(x, w, b):
    projection = x @ w + b
    return projection

In [9]:


np.random.seed(4321)
x = np.random.rand(3,2)
w = np.random.rand(2,3)
b = np.random.rand(3,1)
lp = linear_projection(x, w, b)
print(lp)

# ->

# [[0.49964645 0.7764272  0.59947811]
#  [1.0642018  1.42264665 0.86367775]
#  [1.06047186 1.43087917 1.14610938]]


[[0.49964645 0.7764272  0.59947811]
 [1.0642018  1.42264665 0.86367775]
 [1.06047186 1.43087917 1.14610938]]


In [10]:
def multi_head_attention(x, attn, number_of_heads):
    w_1, b_1 = attn["c_attn"]["w"], attn["c_attn"]["b"]
    w_2, b_2 = attn["c_proj"]["w"], attn["c_proj"]["b"]
    mask = (1 - np.tri(x.shape[0], dtype=x.dtype)) * -1e10
    lp = linear_projection(x, w_1, b_1)
    # Seperate Q,K and V matrices
    m = lp.shape[1]
    if np.mod(m,3) == 0:
        d = m/3
        split = np.hsplit(lp,3) 
        Q = split[0]
        K = split[1]
        V = split[2]
    Q_split = np.hsplit(Q,number_of_heads)
    K_split = np.hsplit(K,number_of_heads)
    V_split = np.hsplit(V,number_of_heads)  
    # Perform masked attention over each head
    for i in range(number_of_heads):
        x_new = masked_attention(Q_split[i],K_split[i],V_split[i],mask)
        if i == 0:
            x = x_new
        else:
            x = np.concatenate([x,x_new], axis=1)
    # Second linear projection with w_2 and b_2
    x = linear_projection(x, w_2, b_2)
    return x

In [11]:
np.random.seed(4321)
x = np.random.rand(3,4)
w_1 = np.random.rand(4,12)
b_1 = np.random.rand(3,1)
w_2 = np.random.rand(4,3)
b_2 = np.random.rand(3,1)
attn = {"c_attn": {"w": w_1, "b": b_1}, "c_proj": {"w": w_2, "b": b_2}}
x = multi_head_attention(x, attn, 2)
print(x)

# [[3.4897257  2.74884012 2.6448295 ]
#  [3.15425828 2.46024887 2.34563449]
#  [3.22513764 2.50993895 2.38375606]]

[[3.4897257  2.74884012 2.6448295 ]
 [3.15425828 2.46024887 2.34563449]
 [3.22513764 2.50993895 2.38375606]]


In [13]:
generate("What is the best food there is?")

ERROR:tensorflow:Couldn't match files for checkpoint models\124M\model.ckpt


Fetching checkpoint: 1.00kb [00:00, ?b/s]                                                           
Fetching encoder.json: 1.04Mb [00:01, 874kb/s]                                                      
Fetching hparams.json: 1.00kb [00:00, ?b/s]                                                         
Fetching model.ckpt.data-00000-of-00001: 498Mb [02:23, 3.46Mb/s]                                    
Fetching model.ckpt.index: 6.00kb [00:00, 6.52Mb/s]                                                 
Fetching model.ckpt.meta: 472kb [00:00, 601kb/s]                                                    
Fetching vocab.bpe: 457kb [00:00, 643kb/s]                                                          
Thinking...: 100%|██████████| 40/40 [00:06<00:00,  6.21it/s]


'\n\nThe best food is the one that is good for you.\n\nThe best food is the one that is good for you.\n\nThe best food is the one that is good for'